# TP2 - Gradient Descent

## Introducción

El objetivo de este trabajo práctico es a asimilar los conceptos que hemos visto de gradient descent, basandonos en un set de datos reales y generando el algoritmo estudiado de una manera rústica y facil de entender.
Además vamos a reforzar esto realizando gráficos que nos faciliten la compresión de lo que estamos haciendo.

## Análisis de los datos

Trabajaremos con los mismos datos del TP1, donde se busca comparar el progreso de los tiempos ganadores en las carreras de atletismo de los Juegos Olímpicos a lo largo de los años. 
https://www.kaggle.com/datasets/bonniesindelar/comparing-progress-of-olympic-winning-track-times

Para comenzar vamos a analizar las carreras de 200 metros llanos de la categoria mujeres.

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv(r'C:\Users\Alumno\Documents\IA_Ciencia_Datos\Semestre 4\Aprendizaje Automatico 2\archive\women200.csv')

In [6]:
df.describe()

,Year,Result,Avg time,"""Change"" value"
count,18.000000,18.000000,1.00,1.0
mean,1983.833333,22.488333,22.49,14.1
std,23.233470,0.853321,NaN,NaN
min,1948.000000,21.530000,22.49,14.1
25%,1965.000000,21.827500,22.49,14.1
50%,1982.000000,22.195000,22.49,14.1
75%,2003.000000,22.875000,22.49,14.1
max,2021.000000,24.400000,22.49,14.1


Solo nos interesa quedarnos con el año y los tiempos.

In [7]:
df = df[['Year','Result']].dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    18 non-null     float64
 1   Result  18 non-null     float64
dtypes: float64(2)
memory usage: 432.0 bytes


### visualización de los datos

A continuación escriba el código correspondiente para realizar un gráfico de puntos (scatter plots), donde el eje horizontal sea el año y el eje vertical los tiempos.

In [10]:
pip install plotly


  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [112]:
# Completar el código aquí
import plotly.express as px

px.scatter(df, x=df2['Año'], y=df2['Resultado'],trendline='ols')

### Normalización

Como la relación de tamaños entre los años y los tiempos es muy grande vamos a normalizar nuestra feature para simplificar las cosas

In [27]:
from sklearn.preprocessing import MinMaxScaler
# Completar el código aquí
scaler = MinMaxScaler()
df2 = scaler.fit_transform(df.iloc[:,:1])
df2 = pd.DataFrame(df2).rename(columns={0:'Año'})
df2['Resultado'] = df['Result']
df2

,Año,Resultado
0,0.000000,24.40
1,0.054795,23.70
2,0.109589,23.40
3,0.164384,24.00
4,0.219178,23.00
5,0.273973,22.50
6,0.328767,22.40
7,0.383562,22.37
8,0.438356,22.03
9,0.493151,21.81


# Hipotesis

Ahora crearemos un modelo de ML básico que prediga el valor del target según el valor de x ingresada.

Nuestro modelo de ML va a ser una regresión lineal común 
$$ h(x)=w_0 + w_1 x$$

In [28]:
def h(x, w0, w1):
  return w0 + w1 * x

In [32]:
h(df2['Año'],0,1)

0     0.000000
1     0.054795
2     0.109589
3     0.164384
4     0.219178
5     0.273973
6     0.328767
7     0.383562
8     0.438356
9     0.493151
10    0.602740
11    0.657534
12    0.712329
13    0.767123
14    0.821918
15    0.876712
16    0.931507
17    1.000000
Name: Año, dtype: float64

Hay que tener en cuenta que la fución es muy rústica y no realiza ninguna validación de datos, por eso al utilizarla tenemos que pasarle los datos de manera correcta. Pero nos dá algo de flexibilidad también, ya que si le pasamos $x$ como un valor númerico nos va a devolver otro valor númerico que sería nuestra predicción de $y$ y si le pasamos $x$ como un np.array o una serie de pandas nos va a devolver el mismo formato con cada valor de $y$ correspondiente

## Función de costo

Para saber si nuestros parámetros w0 y w1 son los óptimos para nuestro modelo tenemos que primero definir la función de costo J, vamos a utilizar la siguiente:
$$ J = \frac{1}{2m}  \displaystyle\sum_{i=1}^{m} [ h(x_{i}) - y_{i}]²  $$

In [49]:
# crear función de costo aquí
# x será el conjunto de valores de la feature "Year"
x = df2['Año']
# y será el conjunto de valores del target "Result"
y= df2['Resultado']
# la función debe retornar un valor numérico

def J(x, y, w0, w1):
  return((((h(x,w0,w1)-y)**2).sum())/(2*df2.shape[0]))

In [53]:
J(x,y,0,8)

177.44075901200972

# Gradient Descent

Utilizaremos el algoritmo del decenso del gradiente estudiado para encontrar los valores de w0 y w1 que obtengan el valor mínimo en la función de costo J

## Vector Gradiente

Primero nos vendría bien una función que nos calcule el vector gradiente en un punto cualquiera A(w0,w1) de la función de costo utilizada, recordemos la formula
$$ \nabla J= [\frac{\partial J}{\partial w_0} ,  \frac{\partial J}{\partial w_1} ] $$

$$ \nabla J = [\frac{1}{m}  \displaystyle\sum_{i=1}^{m}  [h(x_{i}) - y_{i}]  , \frac{1}{m}  \displaystyle\sum_{i=1}^{m}  [h(x_{i}) - y_{i}] x_i ]$$   


In [64]:
# Crear la función que devuelva el vector gradiente aquí

def gradient(x, y, w0, w1):
  cord1 = ((h(x,w0,w1)-y).sum())/(df2.shape[0])
  cord2 = (((h(x,w0,w1)-y)*x).sum())/(df2.shape[0])
  return cord1,cord2

In [72]:
gradient(x, y, 1,1)[0]

-20.997465753424656

## Algoritmo

Muy bien, ahora tenemos todo listo para entrenar a nuestro modelo, realicemos el algoritmo de gradient descent. Comencemos con valores arbitrarios de w0=0 w1=0 y alpha=0.01 (recordemos que el alpha nos va a ayudar a no pegar saltos tan grades que hagan que nuestro algoritmo no converja en un mínimo.

Recordemos como era el algoritmo, tenemos que realizar una iteración de n veces arbitrarias (probemos con valores chicos 10, 20) y en cada una de ellas modificar un poco el punto A(w0, w1) para que vayan en la dirección opuesta al crecimiento de la función J.
$$A_n = A_{n-1} - \alpha  \nabla J(A_{n-1}) $$ 

In [78]:
hola = pd.DataFrame(data=[[1,2]])
hola

,0,1
0,1,2


In [120]:
# La función debe devolver un dataframe que contenga los registros ordenados de 
# cada paso realizado con el valor de w0, w1 y el valor de J
from sklearn.metrics import r2_score
def GradientDescent (x, y, alpha=0.1, steps=10):
  w0 = 0
  w1 = 0
  j = J(x, y, w0, w1)
  Result =  pd.DataFrame(data=[{'w0':w0,'w1':w1,'J':j}])
  for times in range(steps): 
    G = gradient(x,y,w0,w1)
    j = J(x, y, w0, w1)
    w0 = w0 - alpha*G[0]
    w1 = w1 - alpha*G[1]
    data=[{'w0':w0,'w1':w1,'J':j}]
    Result = Result.append(data, ignore_index=True)
  return Result

In [142]:
GradientDescent(x,y,alpha=.5,steps=70).tail()

,w0,w1,J
66,23.145936,-1.413802,0.129990
67,23.164130,-1.448054,0.126801
68,23.181633,-1.481006,0.123850
69,23.198472,-1.512707,0.121118
70,23.214672,-1.543206,0.118590


In [143]:
df112 = GradientDescent(x,y,alpha=.5,steps=70)
index = df112['J'].idxmin()
w0 = df112['w0'].iloc[index]
w1 = df112['w1'].iloc[index]
h(x, w0, w1)

0     23.214672
1     23.130113
2     23.045554
3     22.960994
4     22.876435
5     22.791876
6     22.707317
7     22.622757
8     22.538198
9     22.453639
10    22.284520
11    22.199961
12    22.115402
13    22.030843
14    21.946283
15    21.861724
16    21.777165
17    21.671466
Name: Año, dtype: float64

In [139]:
df2

,Año,Resultado
0,0.000000,24.40
1,0.054795,23.70
2,0.109589,23.40
3,0.164384,24.00
4,0.219178,23.00
5,0.273973,22.50
6,0.328767,22.40
7,0.383562,22.37
8,0.438356,22.03
9,0.493151,21.81


## Gráfico

¿Cómo se vería nuestro gradiente descendiendo por nuestra función J? 

Con los datos obtenidos del algoritmo de GradientDescent colocaremos cada punto obtenido en el plano w0,w1 y los uniremos con una linea.

¿Qué pasaria si ajustamos el alpha o la cantidad de iteraciones?

In [ ]:
# Completar código que grafique los puntos A(w0,w1) obtenidos en el algoritmo de GD

¿Cómo se vería nuestro modelo en el gráfico de puntos que realizamos en un comienzo? 

In [ ]:
# Completar código que grafique la recta h(x) sobre el scatter plot de los datos reales